In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-child-seat-sales-b2/sample_submission.csv
/kaggle/input/car-child-seat-sales-b2/train.csv
/kaggle/input/car-child-seat-sales-b2/test.csv


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
df = pd.read_csv('/kaggle/input/car-child-seat-sales-b2/train.csv')
print(df.head())

   ID  totalSales  CompetitorPrice  Income  Advertising  Population  Price  \
0   1        6.01              131      29           11         335    127   
1   2        6.50              148      51           16         148    150   
2   3        7.70              118      71           12          44     89   
3   4        7.78               86      54            0         497     64   
4   5       11.82              113      66           16         322     74   

  Location  Age  Education Urban   US  
0      Bad   33         12   Yes  Yes  
1   Medium   58         17    No  Yes  
2   Medium   67         18    No  Yes  
3      Bad   33         12   Yes   No  
4     Good   76         15   Yes  Yes  


In [3]:
train_data=pd.read_csv("/kaggle/input/car-child-seat-sales-b2/train.csv")
test_data=pd.read_csv("/kaggle/input/car-child-seat-sales-b2/test.csv")

In [4]:
cat=[]
for i in train_data.iloc[:,1]:
    if i<=5:
        cat.append(1)
    elif i>5 and i<=10:
        cat.append(2)
    else:
        cat.append(3)
train_data["Category"]=cat
train_data

,ID,totalSales,CompetitorPrice,Income,Advertising,Population,Price,Location,Age,Education,Urban,US,Category
0,1,6.01,131,29,11,335,127,Bad,33,12,Yes,Yes,2
1,2,6.50,148,51,16,148,150,Medium,58,17,No,Yes,2
2,3,7.70,118,71,12,44,89,Medium,67,18,No,Yes,2
3,4,7.78,86,54,0,497,64,Bad,33,12,Yes,No,2
4,5,11.82,113,66,16,322,74,Good,76,15,Yes,Yes,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,345,9.70,138,61,12,156,120,Medium,25,14,Yes,Yes,2
345,346,5.35,130,58,19,366,139,Bad,33,16,Yes,Yes,2
346,347,11.70,144,69,11,131,104,Medium,47,11,Yes,Yes,3
347,348,8.73,129,76,16,58,121,Medium,69,12,Yes,Yes,2


In [5]:
train_data["Location"].replace(["Good","Medium","Bad"],[0,1,2],inplace=True)
train_data["Urban"].replace(["Yes","No"],[1,0],inplace=True)
train_data["US"].replace(["Yes","No"],[1,0],inplace=True)
train_data

,ID,totalSales,CompetitorPrice,Income,Advertising,Population,Price,Location,Age,Education,Urban,US,Category
0,1,6.01,131,29,11,335,127,2,33,12,1,1,2
1,2,6.50,148,51,16,148,150,1,58,17,0,1,2
2,3,7.70,118,71,12,44,89,1,67,18,0,1,2
3,4,7.78,86,54,0,497,64,2,33,12,1,0,2
4,5,11.82,113,66,16,322,74,0,76,15,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,345,9.70,138,61,12,156,120,1,25,14,1,1,2
345,346,5.35,130,58,19,366,139,2,33,16,1,1,2
346,347,11.70,144,69,11,131,104,1,47,11,1,1,3
347,348,8.73,129,76,16,58,121,1,69,12,1,1,2


In [6]:
y=train_data.iloc[:,12]
y

0      2
1      2
2      2
3      2
4      3
      ..
344    2
345    2
346    3
347    2
348    2
Name: Category, Length: 349, dtype: int64

In [7]:
test_data["Location"].replace(["Good","Medium","Bad"],[0,1,2],inplace=True)
test_data["Urban"].replace(["Yes","No"],[1,0],inplace=True)
test_data["US"].replace(["Yes","No"],[1,0],inplace=True)
test_data

,ID,CompetitorPrice,Income,Advertising,Population,Price,Location,Age,Education,Urban,US
0,1,131,21,8,220,171,0,29,14,1,1
1,2,118,86,7,265,114,0,52,15,0,1
2,3,129,69,10,289,110,1,50,16,0,1
3,4,157,53,0,403,124,2,58,16,1,0
4,5,112,88,2,243,99,1,62,11,1,1
5,6,122,76,0,270,100,0,60,18,0,0
6,7,121,24,0,200,133,0,73,13,1,0
7,8,143,77,25,448,156,1,43,17,1,1
8,9,138,83,0,139,134,1,54,18,1,0
9,10,116,72,0,237,128,0,70,13,1,0


In [8]:
xp=test_data.iloc[:,1:]
xp.head()

,CompetitorPrice,Income,Advertising,Population,Price,Location,Age,Education,Urban,US
0,131,21,8,220,171,0,29,14,1,1
1,118,86,7,265,114,0,52,15,0,1
2,129,69,10,289,110,1,50,16,0,1
3,157,53,0,403,124,2,58,16,1,0
4,112,88,2,243,99,1,62,11,1,1


In [9]:
x=train_data.iloc[:,2:12]
x
y=train_data.iloc[:,12]
y

0      2
1      2
2      2
3      2
4      3
      ..
344    2
345    2
346    3
347    2
348    2
Name: Category, Length: 349, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
  
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
  
sc_x = StandardScaler()
xtrain = sc_x.fit_transform(xtrain)
xtest = sc_x.transform(xtest)
  
print (xtrain[0:10, :])

[[ 0.54968355 -1.29119825 -0.89222178 -1.28561632  1.25818441 -0.02340182
  -0.72956133  1.5837461   0.68889496  0.71933432]
 [-0.44575379  1.77759374  1.06974096  1.15090616  0.22526377 -0.02340182
  -1.35841044  0.45332386  0.68889496  0.71933432]
 [-0.44575379 -0.90319007  0.46606011  0.65692626  0.26658059  1.50356686
  -1.79860481  0.07651645  0.68889496  0.71933432]
 [-0.32132412 -0.02135329  1.52250159 -0.81166263  0.9276498  -0.02340182
   0.0879425  -1.05390579  0.68889496  0.71933432]
 [ 0.54968355  1.24849167  1.06974096  0.10286772  0.30789742 -1.5503705
   0.46525196  1.20693868  0.68889496  0.71933432]
 [-0.25910929  0.08446712  1.22066117  1.27773883 -0.18790449 -0.02340182
   1.15698597  0.45332386  0.68889496  0.71933432]
 [ 0.61189838 -1.36174519  0.76790053 -0.22422708 -0.22922132 -1.5503705
   0.52813687  1.5837461  -1.4516001   0.71933432]
 [-0.13467962  0.71938961  0.01329948 -1.5793341   1.83661997  1.50356686
  -1.60995008  1.20693868  0.68889496  0.71933432]
 [

In [12]:
from sklearn.linear_model import LogisticRegression
  
classifier = LogisticRegression(random_state = 42,solver="liblinear")
classifier.fit(xtrain, ytrain)

LogisticRegression(random_state=42, solver='liblinear')

In [13]:
y_pred = classifier.predict(xtest)
y_pred

array([2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2,
       2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2,
       2, 2, 3, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2])

In [14]:
from sklearn.metrics import accuracy_score
  
print ("Accuracy : ", accuracy_score(ytest, y_pred))

Accuracy :  0.7386363636363636


In [15]:
from sklearn.preprocessing import StandardScaler
stdx = StandardScaler()
xp = stdx.fit_transform(xp)

print (xp[0:10, :])

[[ 1.52276253e-01 -1.74381793e+00  3.61222463e-01 -3.88673024e-01
   1.97414901e+00 -1.30107648e+00 -1.36647949e+00 -6.84059653e-02
   6.45497224e-01  9.42809042e-01]
 [-8.27909727e-01  5.69409935e-01  2.21659239e-01 -2.62168720e-02
  -3.34318879e-01 -1.30107648e+00 -4.28787950e-02  3.19227838e-01
  -1.54919334e+00  9.42809042e-01]
 [ 1.47841023e-03 -3.55881210e-02  6.40348912e-01  1.67093076e-01
  -4.96316625e-01  1.41421356e-01 -1.57974508e-01  7.06861641e-01
  -1.54919334e+00  9.42809042e-01]
 [ 2.11264821e+00 -6.04998056e-01 -7.55283332e-01  1.08531533e+00
   7.06754874e-02  1.58391919e+00  3.02408344e-01  7.06861641e-01
   6.45497224e-01 -1.06066017e+00]
 [-1.28030326e+00  6.40586177e-01 -4.76156883e-01 -2.03417657e-01
  -9.41810428e-01  1.41421356e-01  5.32599769e-01 -1.23130737e+00
   6.45497224e-01  9.42809042e-01]
 [-5.26314041e-01  2.13528726e-01 -7.55283332e-01  1.40560338e-02
  -9.01310991e-01 -1.30107648e+00  4.17504056e-01  1.48212925e+00
  -1.54919334e+00 -1.06066017e+00

In [16]:
yp=train_data.iloc[:51,12]
yp
classifier = LogisticRegression(random_state = 0)
classifier.fit(xp,yp)
fin_pred = classifier.predict(xp)
fin_pred

array([2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 3, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2])

In [17]:
Y=pd.read_csv("/kaggle/input/car-child-seat-sales-b2/sample_submission.csv",usecols=['ID'])['ID'].tolist()
print(Y)
final = pd.DataFrame({'ID': Y, 'Category': fin_pred})
final.to_csv('sample_submission.csv',index=False)
final

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


,ID,Category
0,1,2
1,2,2
2,3,2
3,4,1
4,5,2
5,6,2
6,7,2
7,8,2
8,9,1
9,10,2
